In [ ]:
# setup
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


import sys
import os
sys.path.append(os.path.abspath(".."))

#helper function
from Helper_funtions import lade_excel

# load data
pfad = "c:/Users/tills/Bachelor-Code/Analyse_Files/Data/Fertige Tabelle.xlsx"
df = lade_excel(pfad)

# Distribution: Gender
sns.countplot(data=df, x="Geschlecht")
plt.title("Geschlechterverteilung")
plt.xlabel("Geschlecht")
plt.ylabel("Anzahl")
plt.show()

# Distribution: age
sns.histplot(data=df, x="Alter", bins=10, kde=True)
plt.title("Altersverteilung")
plt.xlabel("Alter")
plt.ylabel("Anzahl")
plt.show()

# Distribution: educational level
sns.countplot(data=df, x="Bildungsniveau")
plt.title("Verteilung des Bildungsniveaus")
plt.xlabel("Schulstufe")
plt.ylabel("Anzahl")
plt.xticks(rotation=30)
plt.show()

# Distribution: educational level parents
sns.countplot(data=df, x="Bildung Eltern", order=df["Bildung Eltern"].value_counts().index)
plt.title("Bildung der Eltern")
plt.xlabel("Höchster Abschluss")
plt.ylabel("Anzahl")
plt.xticks(rotation=30)
plt.show()

# preferred/least preferred subjects

for fachtyp in ["Lieblingsfach", "Hassfach"]:
    sns.countplot(data=df, x=fachtyp, order=df[fachtyp].value_counts().index)
    plt.title(f"{fachtyp} – Verteilung")
    plt.xlabel("Fach")
    plt.ylabel("Anzahl")
    plt.xticks(rotation=45)
    plt.show()

# Distribution CRT-Scores
sns.histplot(df["crt_score"], discrete=True)
plt.title("Verteilung des CRT-Scores")
plt.xlabel("CRT-Score (0–3)")
plt.ylabel("Anzahl")
plt.show()


In [ ]:
# Setup
import os
import sys
import pandas as pd

# Add helper function path
sys.path.append(os.path.abspath(".."))
from Helper_funtions import lade_excel, clean_up_subjects, analyze_subject_distribution, analyze_distribution

# Load data
file_path = "c:/Users/tills/Bachelor-Code/Analyse_Files/Data/Fertige Tabelle.xlsx"
df = lade_excel(file_path)
df.columns = df.columns.str.strip()

# ---- Demographic Analysis ----

# Gender
analyze_distribution(df, "Gender", "Gender Distribution")

# Age
analyze_distribution(df, "Age", "Age Distribution")

# Education Level
analyze_distribution(df, "Education Level", "Students’ Education Level")

# Hours per week
analyze_distribution(df, "Hours per week for school", "Hours per week distribution")

# Parents' Education Levels
for parent_col in ["Educational Level parent_1", "Educational Level parent_2"]:
    analyze_distribution(df, parent_col, f"Parental Education – {parent_col}")

# CRT Score
analyze_distribution(df, "CRT_points", "Distribution of CRT Score")


# ---- Subject Preferences ----

# Clean up subject columns before analyzing
subject_columns = ["Preferred Subjects", "Least preferred Subjects"]
for col in subject_columns:
    df = clean_up_subjects(df, col)

# Favorite subjects
analyze_subject_distribution(df, "Preferred Subjects")

# Least favorite subjects
analyze_subject_distribution(df, "Least preferred Subjects")
